In [3]:
# need to use updated master JupiterCVML code
import sys
sys.path.append('/home/li.yu/code/JupiterCVML/europa/base/src/europa')

import os
os.environ["BRT_ENV"] = 'prod'
import json
import ndjson
import random
import brtdevkit
print(brtdevkit.__version__)
brtdevkit.log = 'info'
import os
os.environ['AWS_PROFILE'] = 'default'

import time
import numpy as np
import pandas as pd
from datetime import datetime, timezone

from brtdevkit.core.db.athena import AthenaClient, Table
from brtdevkit.data import Image, Dataset

from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
from aletheia_dataset_creator.config.dataset_config import *

pd.set_option('display.max_columns', None)

%matplotlib inline
%load_ext autoreload
%autoreload 2

7.4.3
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print(LEFT_CAMERAS)
print(ALL_CAMERA_PAIRS_LIST)

['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I03', 'I05', 'I07']
[{'front-center-left': 'front-center-right', 'front-left-left': 'front-left-right', 'front-right-left': 'front-right-right', 'side-left-left': 'side-left-right', 'side-right-left': 'side-right-right', 'rear-left': 'rear-right', 'front-center-right': 'front-center-left', 'front-left-right': 'front-left-left', 'front-right-right': 'front-right-left', 'side-left-right': 'side-left-left', 'side-right-right': 'side-right-left', 'rear-right': 'rear-left'}, {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16'}, {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15'}, {'I01': 'I02', 'I03': 'I04', 'I05': 'I06', 'I07': 'I08'}]


In [3]:
# test_dataset = Dataset.retrieve(id='64ed2657926aefcd654e8269')
test_dataset = Dataset.retrieve(name='dynamic_manny_in_dust_raw')
print(test_dataset.id)
test_df = test_dataset.to_dataframe()
test_df.shape

64fa00ea7b198e7e8de6596d


In [4]:
test_df[['id', 'camera_location']].groupby('camera_location').count()

(151412, 111)

,id
camera_location,
front-center-left,13502
front-center-right,13502
front-left-left,12790
front-left-right,12790
front-right-left,13624
front-right-right,13624
rear-left,11472
rear-right,11472
side-left-left,12181


## Run query

In [40]:
# hard_drive_name = 'JUPD-048_2022-09-27'
start_datetime  = datetime(2021, 12, 16, hour=22, minute=14, second=0) #, tzinfo=timezone.utc)
end_datetime  = datetime(2021, 12, 16, hour=22, minute=14, second=35) #, tzinfo=timezone.utc)
# start_datetime  = datetime(2022, 9, 28, tzinfo=timezone.utc) #, hour=3, minute=33, second=33) #, tzinfo=timezone.utc)
# start_datetime  = datetime(2023, 8, 28, hour=0, minute=0)   #, tzinfo=timezone.utc)
# end_datetime    = datetime(2023, 8, 30, hour=23, minute=59)  # this is EXclusive

database = "mesa-data-catalog-prod"
table = "image_jupiter"

query = \
f"""
SELECT *
FROM {table} T
WHERE 
    T.collected_on >= cast('{start_datetime}' as timestamp)
    AND T.collected_on < cast('{end_datetime}' as timestamp)
    AND T.robot_name = 'loamy_733'
"""

"""
SELECT
        *
    FROM annotation_jupiter
    WHERE
     annotation_jupiter.image IN ('642607df1620247e2285d6ac')
"""

"""
SELECT id, camera_location, operation_time, collected_on, state, county, operating_field_name, hdr_mode, robot_name, special_notes
T.collected_on >= cast('{start_datetime}' as timestamp)
    AND T.collected_on < cast('{end_datetime}' as timestamp)
    AND T.camera_location LIKE 'front-%'
    AND T.camera_location LIKE '%-left'
    AND T.hard_drive_name = 'JUPD-154_2023-01-14'
    AND T.robot_name LIKE 'loamy%'
    AND T.operation_time != 'daytime'
    AND T.has_nearby_stop_event = true
ORDER BY RAND()
LIMIT 40000
"""

start = time.time()
athena = AthenaClient()
df = athena.get_df(query)
end = time.time()
print(end - start, 's')
df.shape, len(df) // 2

28.501166105270386 s


((840, 253), 420)

In [41]:
id = '61c40f6d645b3a75242fb494'
print(df[df.id == id].shape)
sub_df = df[df.camera_location.str.startswith('rear')]
print(sub_df.shape)
ids = sub_df[sub_df.camera_location == 'rear-left'].sort_values('collected_on').id.to_list()
i = ids.index(id)
print(f'{id} index {i} out of {len(ids)} images')
for j in range(max(0, i-3), min(i+4, len(ids))):
    print(ids[j])

(1, 253)
(142, 253)
61c40f6d645b3a75242fb494 index 51 out of 71 images
61c41049e5100b0f858cc85b
61c40fc732160727091887d5
61c41029ae6321c259440152
61c40f6d645b3a75242fb494
61c40f7e6b0c478c85db4282
61c41059ec85fdd36b953ac2
61c40ff840a09b978c74a5ac


In [45]:
df[['id', 'camera_location']].groupby('camera_location').count()

,id
camera_location,
front-right-left,49
front-right-right,49
rear-left,71
rear-right,71
side-right-left,88
side-right-right,88


In [43]:
# dfs = []
dfs.append(sub_df)
print(len(dfs))

3


In [44]:
df = pd.concat(dfs, ignore_index=True)
df.shape

(416, 253)

## Create a dataset

In [3]:
# df = pd.read_csv('./dust_threshold/human_vehicle_in_dust_for_train_total.csv')
# df = pd.read_csv('./dust_threshold/dust_productivity.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/driveable_not_labeled.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/less_than_half_labeled.csv')
# df = pd.read_csv('/data/jupiter/datasets/Jupiter_train_v5_11/less_than_half_or_driveable_not_labeled_for_relabeling.csv')
df = pd.read_csv('/data/jupiter/li.yu/exps/driveable_terrain_model/rgb_baseline_sample_a_v3_2/20230925_halo_rgb_stereo_train_v3_epoch0/pruned_ids.csv')
df.shape

(180, 2)

In [46]:
# unlabeled dataset
Dataset.create(
    name="Jupiter_human_on_path_3_fn_sequence",
    description="3 sequences containing 3 common FNs in human on path test set",
    kind='image',
    image_ids=df.id.to_list())

{}

In [26]:
imageids_to_dataset(
    image_ids=test_df[test_df.camera_location.str.endswith('left')].id.to_list(),
    dataset_name="20230823_labeled_right_images_12k_left_images_labels",
    dataset_description="left images and labels of 20230823_labeled_right_images_12k",
    dataset_kind='annotation',  # annotation or image
    mode='mono',  # stereo or mono
    camera_location=CORE_LEFT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
)
imageids_to_dataset(
    image_ids=test_df[test_df.camera_location.str.endswith('right')].id.to_list(),
    dataset_name="20230823_labeled_right_images_12k_right_images_labels",
    dataset_description="right images and labels of 20230823_labeled_right_images_12k",
    dataset_kind='annotation',  # annotation or image
    mode='mono',  # stereo or mono
    camera_location=CORE_RIGHT_CAMERAS,  # CORE_LEFT_CAMERAS or CORE_RIGHT_CAMERAS or ALL_CORE_CAMERAS
)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Filtered 308 annotations.
Sending 12166 annotated_ids for creating dataset
Time taken to prepare data for dataset creation job: 2.07 mins
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Filtered 0 annotations.
Sending 12166 annotated_ids for creating dataset
Time taken to prepare data for dataset creation job: 2.15 mins


In [4]:
# # in master branch halo datasets
# annotations = imageids_to_annotation_df(
#     image_ids=df.id.to_list(),
# )

imageids_to_dataset(
    image_ids=df.id.to_list(),
    dataset_name="20230925_halo_rgb_stereo_train_v3_high_focal_loss",
    dataset_description="180 images with >0.5 focal loss from dataset 20230925_halo_rgb_stereo_train_v3",
    dataset_kind='annotation',  # annotation or image
    mode='stereo',  # stereo or mono
)
# Dataset.create(
#     name="20230925_halo_rgb_stereo_train_v3_pruned",
#     description="180 images with >0.5 focal loss from dataset 20230925_halo_rgb_stereo_train_v3",
#     kind=Dataset.KIND_ANNOTATION,
#     image_ids=df.id.to_list())

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Filtered 4 annotations.
Warning 38 images do not have a corresponding annotation.
Preparing stereo dataframe...
Size of left dataframe: 129
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Size of stereo dataframe: 129
Preparing stereo dataframe...
Size of left dataframe: 90
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 32 workers.
INFO: Pandaralle

## Download a dataset

In [21]:
dataset_name = 'left_and_right_labels_samples_only_right'
# dataset_dir = os.path.join('/data/jupiter/datasets', dataset_name)
dataset_dir = os.path.join('/data/jupiter/li.yu/data', dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

test_dataset = Dataset.retrieve(name=dataset_name)
test_df = test_dataset.to_dataframe()
test_dataset.download(dataset_dir, df=test_df)

2023-08-30 20:15:24,701 - Dataset - INFO - start downloading dataset: 64f004ac766c28f156388ddc into /data/jupiter/li.yu/data/left_and_right_labels_samples_only_right
100%|██████████| 70/70 [00:01<00:00, 60.76it/s] 
2023-08-30 20:15:42,680 - Dataset - INFO - finished downloading dataset: 64f004ac766c28f156388ddc
